# The IPAC Exoplanet Open-Source Imaging Mission Simulator User-Interface (EXOSIMS-UI) Development Guide

Written by: Rahul I. Patel

<hr>

## 1. Intro

EXOSIMS is an all-in-one sophisticated python-based simulation software that provides planetary yield estimates for space-based coronagraphic missions. EXOSIMS incorporates Monte-Carlo completeness modeling, simulates a planet-finding survey given a target list and other contraints, and optimizes the observing time during a given mission lifetime. 
 
IPAC is one of the science center for the Wide-Field Infrared Survey Telescope (WFIRST) mission, and is hosting various tools to help the community plan their observations. One of those include the coronagraphic instrument (CGI) WFIRST. To make EXOSIMS accessible to the community, we are hosting it as a backend to our front end web user interface (UI). 

## 1.1 What You Need

Here are the things you'll need to get just EXOSIMS up and running:

- EXOSIMS full code : 
> https://github.com/dsavransky/EXOSIMS. Useful to get one of the official releases https://github.com/dsavransky/EXOSIMS/releases

- Forecaster : (http://ascl.net/1701.007).  The $\texttt{fitting_parameters.h5}$ from https://github.com/chenjj2/forecaster  should be placed in $\texttt{PlanetPhysicalModel}$.
- You need an SPK ephemeris file and place it in the $\texttt{Observatory}$ subdirector of EXOSIMS. Download it form here: http://naif.jpl.nasa.gov/pub/naif/generic_kernels/spk/planets/de432s.bsp
- [Optional] NASA's Navigation and Ancillary Information Facility's SPICE system components (https://naif.jpl.nasa.gov/naif/)
- Other dependencies: https://github.com/dsavransky/EXOSIMS/blob/master/.travis.yml
- Python requirements: https://github.com/dsavransky/EXOSIMS/blob/master/requirements.txt

** The code at the moment is Python 2 compliant, and they're working on getting up to Python 3. I can't guarantee it'll work in Py3 environment**

## 1.2 Install and Quick Start

The official EXOSIMS github page has two pages to help with this. You can find them here:

- https://github.com/dsavransky/EXOSIMS/blob/master/documentation/install.rst
- https://github.com/dsavransky/EXOSIMS/blob/master/documentation/quickstart.rst

***

## 2. EXOSIMS

  There is a lot to this code. There are too many functions to cover. I dont' intend to outline all of them here, as that would be impractical. There is already documentation for each and every function and variable in the EXOSIMS Internal Control Document (ICD; https://cdn.rawgit.com/dsavransky/EXOSIMS/master/ICD/icd.pdf). 
   
   Here, my goal is to highlight the major working aspects of the code and also the interface script used for the EXOSIMS-UI

### 2.1 The Modules

The code is heavily modular, and various modules control different aspects of the simulation:

- **StarCatalog:** Details on potential targets drawn from general databases. Meta data storage. No other calculations

- **PlanetPopulation:** Encodes the probability density functions of all required planetary parameters, both physical and orbital. Does not model physical orbits or light emission/reflection -- statistics of planet occurence and properties.

- **PlanetPhysicalModel:** Models of the light emitted or reflected by planets in the wavelength bands under investigation by the current mission simulation.

- **OpticalSystem:** Contains all of the necessary information to describe the planet signal and the background noise, and calculate the integration time for a given observation. Encodes specs on instruments. 

- **ZodiacalLight:** Self-explanatory

- **BackgroundSources:** Provides density of background sources for a given target based on its coordinates and the integration depth.

- **PostProcessing:** Encodes the effects of post-processing on the data gathered in a simulated observation, and the effects on the final contrast of the simulation and is also responsible for determining whether a planet detection has occurred for a given observation.

- **Completeness:** Takes in information from the $\texttt{PlanetPopulation}$ module to determine initial completeness and update completeness values for target list stars when called upon. 

 ***
 
 The next few modules are top-level modules that oversee major aspects of the simulation and call upon the powers of the previous modules.
 
 > **TargetList:** Don't let its name fool you. This module takes in information from the $\texttt{StarCatalog}$, $\texttt{OpticalSystem}$, $\texttt{ZodiacalLight}$, $\texttt{PostProcessing}$, $\texttt{BackgroundSources}$, $\texttt{Completeness}$, $\texttt{PlanetPopulation}$, and $\texttt{PlanetPhysicalModel}$  modules to generate the target list for the simulated survey. The final target list encodes all of the same information as is provided by the $\texttt{StarCatalog}$ module.
 
 > **SimulatedUniverse:** Instantiates the $\texttt{TargetList}$ module and creates a synthetic universe by populating planetary systems about some or all of the stars in the target list.
 
 > **Observatory:** Contains all of the information specific to the space-based observatory not included in the $\texttt{OpticalSystem}$ module. 
 
 > **TimeKeeping:** The Time Keeping module is responsible for keeping track of the current mission time. It encodes only the mission start time, the mission duration, and the current time within a simulation.  
 
 > **SurveySimulation:** Performs a specific simulation based on all of the input parameters and models. This module returns the mission timeline - an ordered list of simulated observations of various targets on the target list along with their outcomes.
 
 > **SurveyEnsemble:** only task it to run multiple simulations. Parallel stuff. 

***

Note! The descriptions above are not complete. For a full description, please see the EXOSIMS ICD (https://cdn.rawgit.com/dsavransky/EXOSIMS/master/ICD/icd.pdf)




### 2.2 The Underbelly Process

The way the code works can be outlined by a flow-chart I made back in late 2016 (next Figure). The core of it is still valid, even with updates to the code. 

You'll see that the $\texttt{MissionSimulation}$ starts things up by instantiating the top level modules:

- SurveyEnsemble
- TimeKeeping
- Observatory
- SurveySimulation
- Simulated Universe

After it's been fed a list of user defined inputs from a JSON file. We'll come back to the JSON file shortly.

Then $\texttt{SimulatedUniverse}$ instantiates $\texttt{TargetList}$, which itself in turn instantiates the rest of the major modules. 

![title](../FlowCharts/EXOSIMS_MissionSim.png)

 Running EXOSIMS is relatively simple. As shown in the above flow chart, the $\texttt{MissionSim}$ module is instantiated with an input parameter/script file, and then the $\texttt{SurveySimulation}$ module from $\texttt{MissionSim}$ is executed and the simulation starts running. 

The simulation then runs according to the next Flow Chart figure. Again, it's slightly out of date, but you can get the idea. 

![title](../FlowCharts/EXOSIMS_runsim.png)

## 2.3 The JSON Script

The starting point of EXOSIMS lies in its input scriptfile (i.e., parameter file).
This is what is needed for the user to specify their own simulation parameters and is fed directly into the instantiation of $\texttt{MissionSim}$. 

I would break up the JSON script into five separate sections:
- General Parameters
- Science Instruments
- Star light suppression systems
- Observing Modes
- Modules

You can add as many of these as you want of the parameters that are designated as general parameters. You may want to keep determine which parameters should stay fixed, compared to the ones the user can play around with. Any parameters that are not explictly defined will be given the default values (listed in the ICD).

![title](../scripts/sample_JSONScript.png)

### 2.3.1 General Parameters

I usually include these at the top of the file as they don't belong to any subgroup of parameters. Here are some to name a few, but the majority can be found the ICD

    "logfile":"logfile.txt",    logfile name if you wanna give it one.
    "missionLife": 4,           Life of the mission in years. 
    "missionPortion" : 0.1 ,    Portion of the missison devoted to coronagraphic observations.
    "missionStart" : 60676.0,   Modified Julian Date of the start of the mission.
    "extendedLife" : 0,         How many years to extend original mission.
    "pupilDiam" : 2.37,         Pupil Diameters
    "shapeFac" :0.7854,         Telescope aperture shape factor
    "obscurFac" : 0.14728,      Fraction of primary obscured
    "minComp" : 0.1,            The minimum completeness level for each target to be included in the survey.
    "settlingTime" : 1,         Amount of time, in units of days, needed for observatory to settle after a repointing.
    "dMagLim" :23,              Fundamental limiting magnitude (contrast) difference between the star and planet to which the the mission will be sensitive.
    "intCutoff" :50 ,           Maximum integration time for any obs in days
    "FAP" : 3E-7,               Planet detection false alarm probability.
    "MDP" : 1E-3,               Probability that a planet detection was missed.
    

### 2.3.2 Science Instruments

Pretty self-explanatory. You can define as many instruments you want with a multitude of specifications.

    "scienceInstruments": [
                            {
                              "name": "imaging-EMCCD",
                              "QE": 0.88,
                              "CIC": 0.0013,
                              "ENF": 1.414,
                              "pitch": 1E-5,
                              "focal": 100,
                              "idark": 5E-4,
                              "sread": 0.2, 
                              "texp": 1000
                            },
                            {
                              "name": "spectro-CCD",
                              "QE": 0.88,
                              "CIC": 0.0013,
                              "sread": 3,
                              "Rs": 70,
                              "pitch": 1E-5,
                              "focal": 100,
                              "idark": 5E-4,
                              "texp": 1000 
                            }
                         ],

### 2.3.3. Star Light Suppression Systems

Describes the occulting masks and their specs. These are encoded as a list of dictionaries containing specific attributes of all starlight suppression systems. For each system, if all attributes are missing from the dictionary, they will be assigned the default values listed, or any value directly passed as input to the class constructor. In case of multiple systems, specified wavelength values (lam, deltaLam,BW) of the first system become the new default values. All attributes can be found in the ICD.

    "starlightSuppressionSystems": [
                                        {
                                          "name": "HLC-565",
                                          "lam": 565,
                                          "BW": 0.10,
                                          "IWA": 0.13,
                                          "OWA": 1.44,
                                          "occ_trans": 0.2,
                                          "core_thruput": 0.02,
                                          "core_mean_intensity": 2e-12
                                        },
                                        {
                                          "name": "SPC-800",
                                          "lam": 800,
                                          "BW": 0.18,
                                          "IWA": 0.17,
                                          "OWA": 0.64,
                                          "occ_trans": 0.3,
                                          "core_thruput": 0.04,
                                          "core_mean_intensity": 4e-12,
                                          "core_area": 0.005
                                        }
                                    ],

### 2.3.4. Observing Modes

List of dictionaries containing specific attributes of all mission observing modes. Each observing mode is a combination of an instrument and a system, operating at a given wavelength, which by default is the wavelength defined in the starlight suppression system of the observing mode. If an observing mode is operating at a different wavelength than the system default wavelength, then this new wavelength must be added to the observing mode, and the system performance will be automatically rescaled to the new wavelength. If no observing mode is defined, the default observing mode simply combines the first instrument and the first system.

Details are in the ICD

    "observingModes": [
                        {
                          "instName": "imaging-EMCCD",
                          "systName": "HLC-565",
                          "SNR":5,
                          "lam": 565,
                          "BW": 0.10,
                          "detection":true
                        },

                        {
                          "instName": "spectro-CCD",
                          "systName": "SPC-800",
                          "lam": 400,
                          "BW": 0.10,
                          "SNR":5
                        }
                    ],

### 2.3.5 Modules

Indicates which specific modules will be used for to calculate the specifics of each "universe". If you leave all of them blank, it'll run, but it will default to non-sense. StarCatalog should have something in it besides "StarCatalog". The various modules will call the specified sub-module to do the proper calculation for a specific simulation. For instance, the $\texttt{Observatory}$ module, which handles all the physical properties of the observatory (specs, thrust, fuel, etc.) will now rely on the specifics in the $\texttt{WFIRSTObservatoryL2}$ sub-module. If we wanted a Hab-Ex like observatory, we'd create another sub-module and replace it with that.

     "modules": {
                "PlanetPopulation": "KnownRVPlanets",
                "StarCatalog": "EXOCAT1",
                "OpticalSystem": "Nemati",
                "ZodiacalLight": "Stark",
                "BackgroundSources": "GalaxiesFaintStars",
                "PlanetPhysicalModel": "Forecaster",
                "Observatory": "WFIRSTObservatoryL2",
                "TimeKeeping": " ",
                "PostProcessing": " ",
                "Completeness": "GarrettCompleteness",
                "TargetList": "KnownRVPlanetsTargetList",
                "SimulatedUniverse": "KnownRVPlanetsUniverse",
                "SurveySimulation": " ",
                "SurveyEnsemble": " "
               }

## 2.4 Running EXOSIMS

In [1]:
import numpy as np
import json
import EXOSIMS.MissionSim as msim
np.seterr(all='ignore'); # this is to silence all the annoying numpy warnings

### Now we instantiate a MissionSim object with the JSON script. 

This can be done by pointing to a JSON script file or loading a JSON file as a python dictionary and feeding that to EXOSIMS in one of the following ways

*** With the string file location:***

`sim = msim.MissionSim('../scripts/template_rpateltest_KnownRV.json')`

*** Or by explictly loading the JSON Script ***

`script = open('../scripts/template_rpateltest_KnownRV.json').read()`

`specs = json.loads(script)`

`sim = msim.MissionSim(**specs) `

In [2]:
# This shows you what python produces for a loaded JSON file. It's just a python dictionary:
script = open('../scripts/template_rpateltest_KnownRV.json').read()
specs = json.loads(script)
specs

{u'attenuation': 0.441,
 u'dMagLim': 20,
 u'dMagint': 20,
 u'intCutoff': 50,
 u'logfile': u'rpatel_log.txt',
 u'minComp': 0.02,
 u'missionLife': 6,
 u'missionPortion': 1.0,
 u'modules': {u'BackgroundSources': u'GalaxiesFaintStars',
  u'Completeness': u'GarrettCompleteness',
  u'Observatory': u'WFIRSTObservatoryL2',
  u'OpticalSystem': u'Nemati',
  u'PlanetPhysicalModel': u'Forecaster',
  u'PlanetPopulation': u'KnownRVPlanets',
  u'PostProcessing': u'PostProcessing',
  u'SimulatedUniverse': u'KnownRVPlanetsUniverse',
  u'StarCatalog': u'EXOCAT1',
  u'SurveyEnsemble': u'SurveyEnsemble',
  u'SurveySimulation': u'SurveySimulation',
  u'TargetList': u'KnownRVPlanetsTargetList',
  u'TimeKeeping': u'TimeKeeping',
  u'ZodiacalLight': u'Stark'},
 u'obscurFac': 0.1024,
 u'observingModes': [{u'BW': 0.2,
   u'SNR': 5,
   u'detectionMode': True,
   u'instName': u'imager',
   u'lam': 600,
   u'systName': u'HLC-565'},
  {u'SNR': 5, u'instName': u'spectro', u'lam': 550, u'systName': u'SPC-800'},
  {u'

### Now let's run the simulation. Let's start by instantiating a MissionSim object

In [3]:
sim = msim.MissionSim('../scripts/template_rpateltest_KnownRV.json')

Logging to 'rpatel_log.txt' at level 'INFO'
Imported SurveySimulation (prototype module) from EXOSIMS.Prototypes.SurveySimulation
Imported KnownRVPlanetsUniverse (specific module) from EXOSIMS.Simulat[...]se.KnownRVPlanetsUniverse
Imported KnownRVPlanetsTargetList (specific module) from EXOSIMS.TargetList.KnownRVPlanetsTargetList
Imported EXOCAT1 (specific module) from EXOSIMS.StarCatalog.EXOCAT1
Imported Nemati (specific module) from EXOSIMS.OpticalSystem.Nemati
Imported Stark (specific module) from EXOSIMS.ZodiacalLight.Stark
Imported PostProcessing (prototype module) from EXOSIMS.Prototypes.PostProcessing
Imported GalaxiesFaintStars (specific module) from EXOSIMS.BackgroundSources.GalaxiesFaintStars
Imported GarrettCompleteness (specific module) from EXOSIMS.Completeness.GarrettCompleteness
Imported KnownRVPlanets (specific module) from EXOSIMS.PlanetPopulation.KnownRVPlanets
Imported Forecaster (specific module) from EXOSIMS.PlanetPhysicalModel.Forecaster
Generating pdf of orbital 

/Users/rpatel/Documents/envs/py27/lib/python2.7/site-packages/numpy/core/fromnumeric.py:664: UserWarning: Warning: 'partition' will ignore the 'mask' of the MaskedArray.
  a.partition(kth, axis=axis, kind=kind, order=order)
/Users/rpatel/Documents/envs/py27/lib/python2.7/site-packages/numpy/lib/function_base.py:4033: RuntimeWarning: Invalid value encountered in median
  r = func(a, **kwargs)


Cached completeness file not found at "/Users/rpatel/Dropbox/Research/WFIRST/EXOSIMS/EXOSIMS/Completeness/KnownRVPlanetsForecaster478a62480ade724486ff6f0af1b153d270f18862023bfc8edec4cc8da1d87be8.acomp".
Generating completeness.
Marginalizing joint pdf of separation and dMag up to dMagMax
Finished marginalization
Completeness data stored in /Users/rpatel/Dropbox/Research/WFIRST/EXOSIMS/EXOSIMS/Completeness/KnownRVPlanetsForecaster478a62480ade724486ff6f0af1b153d270f18862023bfc8edec4cc8da1d87be8.acomp
Cached dynamic completeness array not found at "/Users/rpatel/Dropbox/Research/WFIRST/EXOSIMS/EXOSIMS/Completeness/KnownRVPlanetsForecasterNematiEXOCAT1KnownRVPlanetsTargetList52aed83b7f8276aba3abbef28dfeb24d.dcomp".
Beginning dynamic completeness calculations
Dynamic completeness calculations finished
Dynamic completeness array stored in u'/Users/rpatel/Dropbox/Research/WFIRST/EXOSIMS/EXOSIMS/Completeness/KnownRVPlanetsForecasterNematiEXOCAT1KnownRVPlanetsTargetList52aed83b7f8276aba3abbef28

*** Note, that the completeness file will be saved in the Completeness folder in EXOSIMS ***

### Next run the command to run the simulation

In [4]:
sim.run_sim()

OB1: survey beginning.
  Observation #1, star ind 41 (of 46) with 1 planet(s), mission time: 0.0
OB2: previous block was 1.03 long, advancing 0.0.
  Observation #2, star ind 3 (of 46) with 5 planet(s), mission time: 1.03
OB3: previous block was 1.3 long, advancing 0.0.
  Observation #3, star ind 45 (of 46) with 4 planet(s), mission time: 2.32
   - Detected planet inds [78] (1/4)
OB4: previous block was 1.03 long, advancing 0.0.
  Observation #4, star ind 2 (of 46) with 3 planet(s), mission time: 3.35
OB5: previous block was 1.07 long, advancing 0.0.
  Observation #5, star ind 25 (of 46) with 2 planet(s), mission time: 4.43
OB6: previous block was 1.16 long, advancing 0.0.
  Observation #6, star ind 10 (of 46) with 1 planet(s), mission time: 5.59
OB7: previous block was 1.11 long, advancing 0.0.
  Observation #7, star ind 35 (of 46) with 1 planet(s), mission time: 6.7
OB8: previous block was 4.63 long, advancing 0.0.
  Observation #8, star ind 19 (of 46) with 1 planet(s), mission time: 

## 2.5 Extracting from the simulation

The code does not provide automatic outputs. It is completely pythonic, modular and internal to the simulation. The MissionSim object ```(sim)``` holds all the simulation information and you need to extract the data via the individual sub-modules and resultnant stored information objects. Here I will give you examples of things you can extract. A lot of the parameters and objects listed in the ICD are accessible through the top-level as they are typically global parameters.

### 2.5.1 Modules

As explained before, all the modules, listed in 2.2, are accessible via the top level "sim" object --- once the simulation is complete. The following shows how to access each module. TargetList (TL) is usually used as most of the modules are instantiated from it.

In [5]:
TL = sim.TargetList
SC = sim.StarCatalog
SU = sim.SimulatedUniverse
SSim = sim.SurveySimulation
OS = sim.OpticalSystem
ZL = sim.ZodiacalLight
BS = sim.BackgroundSources
CP = sim.Completeness
PP = sim.PlanetPopulation
PM = sim.PlanetPhysicalModel

### 2.5.2 DRM

The DRM (Design Reference Mission) is a list, where each element in the list contains a dictionary of each observing run: time elapsed, instruments used, observing mode used, star observed, planet detection and status, SNR, working angle, etc. All in all there are (as of v1.34) 37 different parameters that cna be inlcuded in the dictionary for each observing scene in the DRM.

In [ ]:
# TO SEE THE FULL DRM OUTPUT
DRM

The orange box shows the information in the DRM pertaining to the first observation of the star eps Eri, and the meta data involved in the observation. For instance, the star was observed in the first observing block (OB_nb:1), 1.02 days after the survey started (arrival_time). The only planet was a missed detection (det_status). 

***More info on each is listed in the ICD***.

![title](../FlowCharts/DRM_test.png)

### 2.5.3 Full List of Parameters

The command "sim.genOutspec()" will produce a dictionary of ALL the parameters used in the simulation -- default or otherwise. 

In [ ]:
# Dictionary of all the parameters
sim.genOutSpec()

### 2.5.4 Other Extractions

One can access all the attributes of the modules listed in the ICD (I'm pretty sure) after the simulation is complete by using the modules in 2.5.1. Here I will give some examples of things that we may wish to extract and display for the user (and some of these are used in the final python script).

#### Number of stars observed with at least one planet detection

In [7]:
len([DRM[x]['star_ind'] for x in range(len(DRM)) if 1 in DRM[x]['det_status']])

0

#### Evaluates contrast at the coronagraph central wavelength and inner working angle - should return our input constant contrast

In [9]:
sim.OpticalSystem.starlightSuppressionSystems[0]['core_contrast'](sim.OpticalSystem.starlightSuppressionSystems[0]['lam'],sim.OpticalSystem.starlightSuppressionSystems[0]['IWA'])

1.000000082740371e-10

#### Calculated integration time per target

In [10]:
TL.tint0

<Quantity [5.65252616e-06, 3.89758789e-06, 1.06353898e-06, 2.64006719e-06,
           3.02522075e-02, 8.25999828e-04, 4.82188562e-04, 4.08349422e-04,
           8.53793630e-05, 1.59638765e-03, 1.76726497e-06, 3.73086958e-05,
           1.72134737e-04, 8.49472110e-07, 1.38246516e-05, 4.89354466e-06,
           1.26962482e-05, 2.16543556e-06, 1.03507790e-05, 7.12322283e-06,
           1.15368028e-06, 3.43677913e-04, 2.09873731e-04, 2.72170476e-06,
           6.70470212e-05, 2.12631931e-06, 2.04236919e-06, 3.05292519e-06,
           3.43321055e-06, 3.24313836e-06, 1.67046642e-06, 1.14456983e-05,
           2.30508218e-06, 2.00876695e-06, 2.37263247e-08, 1.89843168e-05,
           1.02640900e-05, 9.08217572e-04, 4.07901738e-05, 1.30994198e-03,
           1.68701010e-03, 2.32147404e-07, 1.61551767e-07, 1.84202539e-07,
           2.13994055e-06, 4.20473114e-07] d>

#### Working Angles

In [22]:
OS.WA0, OS.IWA, OS.OWA

(<Quantity 0.3 arcsec>, <Quantity 0.15 arcsec>, <Quantity inf arcsec>)

#### Limiting Magnitudes for each planet

In [24]:
SU.dMag

array([22.68451219, 17.89834844, 20.97661834, 24.98758862, 21.07390144,
       13.99416863, 18.89634413, 13.80521777, 19.13964888, 18.61777745,
       22.01869688, 18.86860493, 20.09743034, 24.30508166, 25.17446818,
       20.24379468, 23.2538133 , 23.47342465, 22.78045661, 18.60049125,
       17.94478005, 17.47677617, 20.18227379, 23.50335294, 19.64156965,
       26.05990364, 22.8471778 , 20.63012055, 22.29402452, 18.40063377,
       19.03984093, 18.97677721, 24.28892251, 19.98652707, 19.83806725,
       17.76810731, 24.46881061, 20.271966  , 16.88938726, 23.4274189 ,
       20.13202977, 21.24744747, 14.83738334, 23.17399568, 27.2018157 ,
       15.84778001, 16.69013177, 21.39873103, 19.58079527, 20.15950103,
       22.09171389, 21.77266835, 18.89194144, 21.91496222, 26.51241123,
       21.92058264, 22.30522402, 21.96252481, 19.14621225, 23.07852199,
       28.16014893, 21.27697792, 21.29665138, 24.20355875, 20.05468993,
       23.89937904, 18.08631363, 12.73363845, 22.20874609, 25.76

## 3. EXOSIMS UI

The user interface at IPAC has three components.

- 1) The public facing user interface.
- 2) The intermediary python script.
- 3) EXOSIMS

Basically, the user inputs the parameters of their choice in the interface (1). The inputs are then parsed by the back-end code to the intermediary python script (2). The python script then calls EXOSIMS internally (3), whose results are then internally stored and created by the python script (4). Select outputs are then sent to the backend IPAC cloud (5), which the user can then access via the interface or emailed results.

***This section will mostly deal with numbers 2 to 5 in the flow chart below ***

![title](../FlowCharts/exosimsui.png)

### 3.1 Interface Parsing

The basics of the interface is self-explanatory -- stuff on website with forms that the user inputs, yada yada. The top level code is a black box to me, and Joan is best to consult with respect to that. The important thing is that once the user hits "submit", the input values are parsed in an array of strings in the form of 

**['path','key1','val1','key2','val2',...,'keyn','valn']**

where the first term, 'path', can be ignored as it's just the local path for where the script is located. Everything else are seen as pairs of keys and values (input field, and the entered value).

E.g.: ['/var/www/wfirst-dev-15-08-07/html/sims/tools/exosimsCGI/bin/EXOSIMS_end2end+outputs.py',
       '-missionLife', '5.5', '-missionPortion', '0.3', '-extendedLife', '0.0', '-missionStart', '60676.0']
       
The tool "format_args" takes this input list of strings and parses out a dictionary of keys and values, while removing the first entry, which is assumed to be the path to wherever the script is stored on the backend machine:

In [11]:
import numpy as np
args = ['/var/www/wfirst-dev-15-08-07/html/sims/tools/exosimsCGI/bin/EXOSIMS_end2end+outputs.py',
        '-missionLife', '5.5', '-missionPortion', '0.3', '-extendedLife', '0.0', '-missionStart', '60676.0', '-dMagLim',
        '20', '-settlingTime', '0.03', '-minComp', '0.1', '-telescopeKeepout', '10.0', '-intCutoff', '50.0', '-eta',
        '0.1', '-ppFact', '0.3', '-FAP', '0.0000003', '-MDP', '0.001', '-TargetType', 'KnownRV', '-Dlam', '550.0',
        '-DBW', '0.1', '-DSNR', '5.0', '-DohTime', '0.1', '-Clam', '600.0', '-CBW', '0.1', '-CSNR', '5.0', '-CohTime',
        '0.1', '-CatType', 'EXOCAT1'] 

def format_args(args):
    # KEEP ALL BUT FIRST ELEMENT OF ARGV.
    invar = np.array(args[1:])
    # REMOVES ALL DASHES
    invar2 = np.array([term.strip('-') for term in invar])
    invar3 = dict(np.resize(invar2, (invar2.size / 2, 2)))
    return invar3

inputVars = format_args(args)
inputVars

{'CBW': '0.1',
 'CSNR': '5.0',
 'CatType': 'EXOCAT1',
 'Clam': '600.0',
 'CohTime': '0.1',
 'DBW': '0.1',
 'DSNR': '5.0',
 'Dlam': '550.0',
 'DohTime': '0.1',
 'FAP': '0.0000003',
 'MDP': '0.001',
 'TargetType': 'KnownRV',
 'dMagLim': '20',
 'eta': '0.1',
 'extendedLife': '0.0',
 'intCutoff': '50.0',
 'minComp': '0.1',
 'missionLife': '5.5',
 'missionPortion': '0.3',
 'missionStart': '60676.0',
 'ppFact': '0.3',
 'settlingTime': '0.03',
 'telescopeKeepout': '10.0'}

#### 3.1.1 Interface to EXOSIMS Keywords

The backend keywords from the interface are not exactly the same as what EXOSIMS uses. The intermediary python script takes the backend user keywords and translates them to what EXOSIMS uses. To ensure this backend-to-exosims mapping, I created an excel sheet that contains information telling Joan what to label each backend keyword to simplify the translation. For the most part, it's a simple dash that's prefixed to the EXOSIMS keyword to create the UI backend keyword.

This excel sheet can be found in '../GUI_Backend/gui_input.xlsx

A snippet of the excel sheet can be found below. It lists:

 - ***Front end***:
     Listed description of the input variable, along with the the default value, units, and value restriction
 
 - ***Back end***:
     The UI generated back end variable name for that entry, which is typically the corresponding EXOSIMS variable name prefixed with a dash (usually)
     
 - ***Alt Text Description***: self-explanatory.

![title](../FlowCharts/gui_input_sheet.png)

### 3.2 Python Script

The python script, "EXOSIMS_end2end+outputs.py"  formats the array of strings from the UI and parses it to EXOSIMS. It then stores the output, and saves a bunch of output CSV files and plots.

Much of the script is self-explantory. Here, I'll explain the parts that are a little more complicated.

#### 3.2.1  Creating a User Specified JSON

EXOSIMS requires a JSON input file name/python dictionary to start the simulation. To allow the user's selection flow directly into our backend, I've compartmentalized multiple template JSON scripts, such that each handles a different part of the various aspects of section 2.3, and then combines all of them into one dictionary to send to EXOSIMS. 

This starts around line 550 in the python script. 

First, three base JSON files are loaded: $\texttt{baseParams.json, baseObservingModes.json, baseModules.json}$

Each contains standard inputs of what's listed in 2.3, and then the user specified values are added/replaced in the python script. First, each file is loaded as a python dictionary. For example

In [2]:
import os,json

def get_specs(jfile):
    script = open(jfile).read()
    specs = json.loads(script)
    return specs


baseFolder = os.path.join(os.path.expanduser('~'), 'Dropbox', 'Research', 'WFIRST', 'EXOSIMSTesting')
scriptFolder = os.path.join(baseFolder, 'scripts')
jfileBase = os.path.join(scriptFolder, 'baseParams.json')

baseParams = get_specs(jfileBase)

In [3]:
baseParams

{u'FAP': 3e-07,
 u'MDP': 0.001,
 u'attenuation': 0.441,
 u'dMagLim': 23,
 u'extendedLife': 0,
 u'intCutoff': 50,
 u'logfile': u'logfile.txt',
 u'minComp': 0.1,
 u'missionLife': 4,
 u'missionPortion': 0.1,
 u'missionStart': 60676.0,
 u'obscurFac': 0.14728,
 u'pupilDiam': 2.37,
 u'scienceInstruments': [{u'CIC': 0.0013,
   u'ENF': 1.414,
   u'QE': 0.88,
   u'focal': 100,
   u'idark': 0.0005,
   u'name': u'imaging-EMCCD',
   u'pitch': 1e-05,
   u'sread': 0.2,
   u'texp': 1000},
  {u'CIC': 0.0013,
   u'QE': 0.88,
   u'Rs': 70,
   u'focal': 100,
   u'idark': 0.0005,
   u'name': u'spectro-CCD',
   u'pitch': 1e-05,
   u'sread': 3,
   u'texp': 1000}],
 u'settlingTime': 1,
 u'shapeFac': 0.7854,
 u'starlightSuppressionSystems': [{u'BW': 0.1,
   u'IWA': 0.1475,
   u'OWA': 0.4917,
   u'core_area': 0.005,
   u'core_mean_intensity': 2e-12,
   u'core_thruput': 0.01,
   u'lam': 565,
   u'name': u'HLC-565',
   u'occ_trans': 0.2,
   u'ohTime': 1},
  {u'BW': 0.18,
   u'IWA': 0.1608,
   u'OWA': 0.517,
   u

where, "get_specs" is already defined in the python script file. As you can see there are default values for some of the EXOSIMS parameters, and some or all or none can be replaced based on the user's input. This particular dictionary, formed from "baseParams" will be the baseline JSON script that the python script uses. Other aspects of the dictionary will be changed on top of this. 

So, from here, the other template JSON files are loaded, according to lines 560-570 

![title](../FlowCharts/json_use2.png)

The other base dictionaries look a like this:


***Observing Modes***

In [6]:
jfileObsMode = os.path.join(scriptFolder, 'baseObservingModes.json')
obsSpecs = get_specs(jfileObsMode)
obsSpecs

{u'HLC_Detection': {u'BW': 0.1,
  u'SNR': 5,
  u'detection': True,
  u'instName': u'imaging-EMCCD',
  u'lam': 565,
  u'systName': u'HLC-565'},
 u'SPC_Characterization': {u'BW': 0.18,
  u'SNR': 10,
  u'instName': u'spectro-CCD',
  u'lam': 660,
  u'systName': u'SPC-660'}}

***Modules***

In [7]:
jfileModules = os.path.join(scriptFolder, 'baseModules.json')
modSpecs = get_specs(jfileModules)
modSpecs

{u'EarthTwin': {u'BackgroundSources': u'GalaxiesFaintStars',
  u'Completeness': u'GarrettCompleteness',
  u'Observatory': u'WFIRSTObservatoryL2',
  u'OpticalSystem': u'Nemati',
  u'PlanetPhysicalModel': u'PlanetPhysicalModel',
  u'PlanetPopulation': u'EarthTwinHabZone2',
  u'PostProcessing': u'PostProcessing',
  u'SimulatedUniverse': u'SimulatedUniverse',
  u'StarCatalog': u'EXOCAT1',
  u'SurveyEnsemble': u'SurveyEnsemble',
  u'SurveySimulation': u'SurveySimulation',
  u'TargetList': u'TargetList',
  u'TimeKeeping': u'TimeKeeping',
  u'ZodiacalLight': u'Stark'},
 u'KeplerLike': {u'BackgroundSources': u'GalaxiesFaintStars',
  u'Completeness': u'GarrettCompleteness',
  u'Observatory': u'WFIRSTObservatoryL2',
  u'OpticalSystem': u'Nemati',
  u'PlanetPhysicalModel': u'FortneyMarleyCahoyMix1',
  u'PlanetPopulation': u'KeplerLike1',
  u'PostProcessing': u'PostProcessing',
  u'SimulatedUniverse': u'KeplerLikeUniverse',
  u'StarCatalog': u'EXOSCAT1',
  u'SurveyEnsemble': u'SurveyEnsemble',
  u

---

Now the baseParam dictionary is used as the template upon which to add the user's desired inputs. Lines 577 to 612 show that below

![title](../FlowCharts/json_use.png)

The variable "useSpecs" will be the final dictionary that will be passed to EXOSIMS. 

This part of the code loops over all the variables in inputVars (section 3.1), and tries to jam them into "useSpecs". Lines 582-587 updates/adds in parameters that are either floats/integer values. 

Lines 594-602 loops over the wavelength, bandwidth and SNR variables in the  starlight suppression system, and updates those values for both the detection and characterization systems indicated (hence the "D" and "C" prefixes, as indicated in the excel sheet in Section 3.1.1)

Lines 605-607 then adds the observing modes to the "useSpecs" dictionary.

The next step is to run the simulation. That's where lines 617-619 comes in. It takes useSpecs and starts the simulation

![title](../FlowCharts/json_use3.png)

#### 3.2.2  Results Returned

The rest of the code is relatively straightforward. Various objects and variables are extracted from the simulation, as shown in section 2.5. If the simulation doesn't visit any stars (i.e., all filtered out), then most of the rest of the code is ignored, and blank plots are made. In addition, CSV files are created for the user to explore all visited stars, planets and their associated metadata. 

***CSV Files ***

There are a number of files that are created by both the backend non-python and this python script. The python script produces two CSV files: "simResults.csv" and "headerinfo.csv". The python script does this starting at line 752.

***"simResults.csv"*** has the input JSON parameters starting at the top of the file. After that, the stars that were observed along with the planets that were detected, along with all their meta data, are listed in column format.

***"headinfo.csv"*** contains the description of each of the header names in "simResults.csv"

***Plots***

The following are all the plots that are currently created by the script. If there are no stars observed, then plots are still saved, but they're simply blank.

Sky plot of stars included in the simulation and then those that are observed.

![title](../GUI_Backend/fig1.png)

Same as the one above, except a visual guide of the order of observations are hown.

![title](../GUI_Backend/fig2.png)

Color magnitude diagram of the same stars in the above plots.

![title](../GUI_Backend/fig3.png)

Contrast curve of planets observed.

![title](../GUI_Backend/fig4.png)